In [ ]:
# helper function that deals with cases where func(expr) = func(sympyPowerExpr), func(sympyAddExpr), func(sympyMultExpr)
function separate_real_imaginary_power_add_mult(expr::SymPy.Sym)
    # if the expanded expression contains toplevel power calculations, the expression must be of the form x^a, so terms = args(expr) = (x, a)
    if func(expr) == func(sympyPowerExpr)
        result = real(expr) + im*imag(expr)

    # if the expanded expression contains toplevel addition, the individual terms must all be products
    elseif func(expr) == func(sympyAddExpr)
        println(expr)
        println("addition!")
        result = 0
        terms = args(expr)
        for term in terms
            println("term = $term")
            # if term is exponential
            if func(term) == func(sympyExpExpr)
                termSeparated = separate_real_imaginary_exp(term)
            # else, further split each product term into indivdual factors
            else
                (constant, factors) = factor_list(term)
                termSeparated = constant
                # separate each factor into real and imaginary parts and collect the product of separated factors
                for (factor, power) in factors
                    factor = factor^power
                    termSeparated = termSeparated * (real(factor) + im*imag(factor))
                end
            end
            println("termSeparated = $termSeparated") 
            # collect the sum of separated term, i.e., product of separated factors
            result = result + termSeparated
        end
        println("result = $result")

    # if the expanded expression contains toplevel multiplication, the individual terms must all be powers
    elseif func(expr) == func(sympyMultExpr)
        println(expr)
        println("multiplication!")
        result = 1
        terms = args(expr)
        for term in terms
            println("term = $term")
            # if term is exponential
            if func(term) == func(sympyExpExpr)
                termSeparated = separate_real_imaginary_exp(term)
            # else, further split each product term into indivdual factors
            else
                (constant, factors) = factor_list(term)
                termSeparated = constant
                # separate each factor into real and imaginary parts and collect the product of separated factors
                for (factor, power) in factors
                    factor = factor^power
                    termSeparated = termSeparated * (real(factor) + im*imag(factor))
                end
            end
            println("termSeparated = $termSeparated") 
            # collect the product of separated term, i.e., product of separated factors
            result = result * termSeparated
        end
        println("result = $result")
    end
    return result
end

function separate_real_imaginary(delta::SymPy.Sym)
    
    # check if delta has one and only one free symbol (e.g., global variable lambda)
    if length(free_symbols(delta)) == 1
        # substitute lambda as x+iy
        expr = subs(delta, lambda, x+im*y)
        # expand the new expression
        expr = expand(expr)
        # if the expansion contains toplevel power calculations, addition, or multiplications, then it can be separated into more than one term
        if func(expr) in [func(sympyPowerExpr), func(sympyAddExpr), func(sympyMultExpr)]
            terms = args(expr)
            println("terms = $terms")
            result = separate_real_imaginary_power_add_mult(expr)
            
        # if the expanded expression is neither of the above, it must be a single term, e.g., x or cos(x)
        else
            println(expr)
            println("single term!")
            # simply separate the single term
            term = expr
            termSeparated = real(term) + im*imag(term)
            result = termSeparated
            println("result = $result")
        end
        result = expand(result)
        return real(result) + im*imag(result)
        
    else
        throw("Delta has more than one variable")
    end
end